# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm), thank goodness we can search for these things.

## Setup: Import what you'll need to search and scrape and Selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

## Starting from `https://arlweb.msha.gov/drs/drshome.htm`, search for every operator with 'dirt' in their name, including abandoned mines.

> - *Tip: If you can't make an element work using name, class or ID, try to use the XPath*

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

In [5]:
operator_search = driver.find_element_by_name("OperSearch")
operator_search.send_keys("dirt")

In [6]:
button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[7]/td[3]/input[1]')
button.click()

## Scrape the results page, saving it as `dirt-operators.csv`

> - *Tip: Think about what each row in your dataset will be, and start by looping through that*
> - *Tip: Printing is cool and good! Print everything! Move it into a dictionary later.*
> - *Tip: If you don't want a row, think about what's in the row that makes it different. You can use an `if` statement or list slicing to skip the ones you aren't interested in.*
> - *Tip: Make sure your dictionary and your loop variable have DIFFERENT NAMES*
> - *Tip: After you've made your dictionary (and printed it, of course), you'll want to add it to your list of rows*
> - *Tip: Be sure to import pandas to convert it to a dataframe*
> - *Tip: Make sure you don't include the index when saving your dataframe*

### Hopefully you know that each `tr` is supposed to be a row of your data. What is the index of the first row element that is actually a result?

> - *Tip: `.text` will help you here.*
> - *Tip: You aren't interesting in annotations or anything, just mines and where they are from*
> - *Tip: Using `print("-----")` will help you keep track of different rows*
> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third. You can use this to skip ahead to the 'good' data if you want*

In [7]:
#find all of the groups of content on the page
all_rows = driver.find_elements_by_tag_name("tr")

for row in all_rows[6:]:
    print("-----------")
    print(row.text)

-----------
ID State Operator Mine Name Type CM* Status Commodity More Info
-----------
3503598
OR  Newberg Rock & Dirt   Newberg Rock & Dirt Surface M  Active  Crushed, Broken Stone NEC 
-----------
1401575
KS  Bender Sand & Dirt   BENDER SAND & DIRT Surface M  Intermittent  Construction Sand and Gravel 
-----------
5001797
AK  Dirt Company   Bush Pilot Surface M  Intermittent  Construction Sand and Gravel 
-----------
2103723
MN  Dirt Doctor Inc   Rock Lake Plant Surface M  Intermittent  Construction Sand and Gravel 
-----------
2103914
MN  Dirt Work Specialists LLC   Astec Plant Surface M  Intermittent  Construction Sand and Gravel 
-----------
4104757
TX  Dirt Works   Portable #1 Surface M  Intermittent  Construction Sand and Gravel 
-----------
0801306
FL  Holley Dirt Company, Inc   River Road Pit Surface M  Active  Sand, Common 
-----------
3901432
SD  Krueger Brothers Gravel & Dirt   PORTABLE SCREENER Surface M  Intermittent  Construction Sand and Gravel 
-----------
3609624
PA 

### Loop through each operator result, printing its name

> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third.*
> - *Tip: You can use list slicing or an `if` statement to skip the non-data row(s). List slicing is probably easier, even if you aren't comfortable with it.*
> - *Tip: or honestly you can use `try` and `except` if you know how it works.*
> - *Tip: Once you have the "right" rows of data, you're going to be looking for a certain tag inside*
> - *Tip: Sometimes you can't say "give me this class," and instead you have to say "give me all of the `div` elements, and then give me the third one."*

In [8]:
all_rows = driver.find_elements_by_tag_name("tr")

for row in all_rows[7:27:]:
    #print(row.text)
    all_operators = row.find_elements_by_tag_name("td")
    print(all_operators[2].text)

Newberg Rock & Dirt  
Bender Sand & Dirt  
Dirt Company  
Dirt Doctor Inc  
Dirt Work Specialists LLC  
Dirt Works  
Holley Dirt Company, Inc  
Krueger Brothers Gravel & Dirt  
M R Dirt  
M.R. Dirt Inc.  
P B Dirt Movers, Inc  
P B Dirt Movers, Inc.  
PB Dirt Movers  
Prescott Dirt, LLC  
R D Blankenship Dirt Work LLC  
Sand & Dirt, Inc  
SIMPSON DIRTWORX LLC  
SIMPSON DIRTWORX LLC  
Spry's Dirt & Gravel, Inc.  
Vogt Dirt Service  


### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [9]:
all_rows = driver.find_elements_by_tag_name("tr")

for row in all_rows[7:27:]:
    #print(row.text)
    all_operators = row.find_elements_by_tag_name("td")
    print(all_operators[0].text)

3503598
1401575
5001797
2103723
2103914
4104757
0801306
3901432
3609624
3609931
1519799
4407379
4407296
0203332
2901986
0801417
4300768
4300776
2302283
2103518


In [10]:
#for row in all_rows[7:27:]:
#    all_operator_rows_dict = {}
#    all_operators = row.find_elements_by_tag_name("td")
#    for operators in all_operators:
#        print(operators.text)
        #row = operator.text
    
#print(all_operator_rows_dict)

 

## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

> - *Tip: Start with an empty dictionary, then add the keys one at a time like we did during class*
> - *Tip: You might want to save all of the cells in a variable, then use indexes to get the second, third, fourth, etc.*
> - *Tip: I know you already skipped a bunch of rows already, but one of them still might be bad! Which one is it? How can you skip it? You might need to slice out some of the end of your list, too. Use `print` to help you debug, or just look at the page closely.*
> - *Tip: Or, if you did the other homework already, `try` / `except` is also an option*

In [11]:
mine_rows = []
for row in all_rows[7:-1]:
    cells = row.find_elements_by_tag_name("td")
    rows_dict = {}
    
    column_id = all_operators[0]
    rows_dict["column_id"] = cells[0].text  

    state_id = all_operators[1]
    rows_dict["state_id"]= cells[1].text

    operator_id = all_operators[2]
    rows_dict["operator_id"] = cells[2].text  
    
    mine_id = all_operators[3]
    rows_dict["mine_id"] = cells[3].text  
    
    type_id = all_operators[4]
    rows_dict["type_id"] = cells[4].text  

    cm_id = all_operators[5]
    rows_dict["cm_id"] = cells[5].text  
    
    status_id = all_operators[6]
    rows_dict["status_id"] = cells[6].text  

    commodity_id = all_operators[7]
    rows_dict["commodity_id"] = cells[7].text  
    
    mine_rows.append(rows_dict)
       
#for mine_row in mine_rows:
#    print(mine_row.text)
                

### Save that to a CSV named `dirt-operators.csv`

In [12]:
df = pd.DataFrame(mine_rows)
df.head()

,cm_id,column_id,commodity_id,mine_id,operator_id,state_id,status_id,type_id
0,M,3503598,"Crushed, Broken Stone NEC",Newberg Rock & Dirt,Newberg Rock & Dirt,OR,Active,Surface
1,M,1401575,Construction Sand and Gravel,BENDER SAND & DIRT,Bender Sand & Dirt,KS,Intermittent,Surface
2,M,5001797,Construction Sand and Gravel,Bush Pilot,Dirt Company,AK,Intermittent,Surface
3,M,2103723,Construction Sand and Gravel,Rock Lake Plant,Dirt Doctor Inc,MN,Intermittent,Surface
4,M,2103914,Construction Sand and Gravel,Astec Plant,Dirt Work Specialists LLC,MN,Intermittent,Surface


In [139]:
df.to_csv("dirt-operators.csv", index=False)

### Open the CSV file and examine the first few.

Make sure you didn't save that extra weird unnamed index column.

In [138]:
df.head(1)

,cm_id,column_id,commodity_id,mine_id,operator_id,state_id,status_id,type_id
0,M,3503598,"Crushed, Broken Stone NEC",Newberg Rock & Dirt,Newberg Rock & Dirt,OR,Active,Surface
